## Our target

```python
class User(Base):
    id: int
    name: str

user = User(id='123', name='john')
print(user.id)
# 123 -> this is an int
print(user.name)
# john -> this is a str

try:
    user.name = 123
except Exception as e:
    print(e)
```

#### \_\_annotations\_\_ ?

In [ ]:
class User:
    id: int
    name: str

In [ ]:
User.__annotations__

#### Let's give it an \_\_init\_\_ method.

In [ ]:
class BaseModel:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

class User(BaseModel):
    id: int
    name: str
        
user = User(id=123, name="john")

In [ ]:
user.id

In [ ]:
user.name

#### There's no type validations!

-

-

#### Introducing Metaclass
* type(class_name, base_classes, attributes)

In [ ]:
class User:
    id=1
    name="john"
User.__dict__

In [ ]:
User2 = type("User", (), {'id':1, 'name':'john'})
User2.__dict__

#### Custom Metaclass

In [ ]:
class MyMetaClass(type):
    def __new__(metacls, class_name, base_classes, attributes):
        print(f"{class_name} has attributes {attributes}")
        print(attributes)
        return super().__new__(metacls, class_name, base_classes, attributes)

In [ ]:
class User3(metaclass=MyMetaClass):
    id=2
    name="john"

### Do you remember descriptors?
### We want to validate data as it is 'set'

In [ ]:
class Descriptor:
    registry = {}
    def __init__(self, name, data_type):
        self.name = name
        self.data = data_type()
        self.data_type = data_type
    def __get__(self, instance, cls):
        return self.data
    def __set__(self, instance, value):
        ## we want to validate data as it is set
        self.data = value

class MyMetaClass(type):
    def __new__(metacls, class_name, base_classes, attributes):
        annotations = attributes.get('__annotations__',{})
        for name, data_type in annotations.items():
            attributes[name] = Descriptor(name=name, data_type=data_type)
        return super().__new__(metacls, class_name, base_classes, attributes)
    
class User3(metaclass=MyMetaClass):
    id:int
    name:str

In [ ]:
User3.__dict__

In [ ]:
user = User3()
user.name = "john"
user.id = 3

In [ ]:
user.name

In [ ]:
user.id

### Let's validate!

In [ ]:
class Descriptor:
    registry = {}
    def __init__(self, name, data_type):
        self.name = name
        self.data = data_type()
        self.data_type = data_type
    def __get__(self, instance, cls):
        return self.data
    def __set__(self, instance, value):
        try:
            self.data = self.data_type(value)
        except:
            raise Exception(f"{instance}.{self.name} is of type {self.data_type}, '{value}' of {type(value)} is given")

class MyMetaClass(type):
    def __new__(metacls, class_name, base_classes, attributes):
        annotations = attributes.get('__annotations__',{})
        for name, data_type in annotations.items():
            attributes[name] = Descriptor(name=name, data_type=data_type)
        return super().__new__(metacls, class_name, base_classes, attributes)
    
class User4(metaclass=MyMetaClass):
    id:int
    name:str

In [ ]:
user = User4()

In [ ]:
user.name = 123
user.name

In [ ]:
user.id = "999"
user.id

In [ ]:
user.id = "can't be cast as int"

### Final touch: the init

In [ ]:
class Descriptor:
    registry = {}
    def __init__(self, name, data_type):
        self.name = name
        self.data = data_type()
        self.data_type = data_type
    def __get__(self, instance, cls):
        return self.data
    def __set__(self, instance, value):
        try:
            self.data = self.data_type(value)
        except:
            raise Exception(f"{instance}.{self.name} is of type {self.data_type}, '{value}' of {type(value)} is given")

class MyMetaClass(type):
    def __new__(metacls, class_name, base_classes, attributes):
        annotations = attributes.get('__annotations__',{})
        for name, data_type in annotations.items():
            attributes[name] = Descriptor(name=name, data_type=data_type)
        return super().__new__(metacls, class_name, base_classes, attributes)
class Base(metaclass=MyMetaClass):
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)
class User5(Base):
    id:int
    name:str

In [ ]:
user = User5(id="1",name=99876)

In [ ]:
user.id


In [ ]:
user.name

In [ ]:
user.id = "something"

### Your turn! Could you make the code work?
```python
class InternalUser(Base):
    first_name = Column(str)
    last_name = Column(str)
    is_active = Column(bool)

user = InternalUser(first_name="john", last_name="peter", is_active = True)
print(user.first_name)
# john
print(user.last_name)
# peter
print(user.is_active)
# True
user.is_active = False
print(user.is_active)
# False
```

In [ ]:
class Column:
    pass


#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  
#  


# Solution

In [ ]:
class Column:
    def __init__(self, data_type):
        self.name = None
        self.data = data_type()
        self.data_type = data_type
    def __get__(self, instance, cls):
        return self.data
    def __set__(self, instance, value):
        if isinstance(value, self.data_type):
            self.data = value
        else:
            raise Exception(f"{instance}.{self.name} is of type {self.data_type}, {value} of {type(value)} is given")

# class MyMetaClass(type):
#     def __new__(metacls, class_name, base_classes, attributes):
#         for attribute_name, attribute in attributes.items():
#             if isinstance(attribute, Column):
#                 attribute.name = attribute_name
#         return super().__new__(metacls, class_name, base_classes, attributes)
    
class Base():
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

class InternalUser(Base):
    first_name = Column(str)
    last_name = Column(str)
    is_active = Column(bool)

In [ ]:
user = InternalUser(first_name="john", last_name="peter", is_active = True)
print(user.first_name)
# john
print(user.last_name)
# peter
print(user.is_active)
# True
user.is_active = False
print(user.is_active)